#### Note: portfolio.py can be found at https://github.com/MarkHendricks/finm-quant-2022/blob/main/cmds/portfolio.py

## Initialisation

In [168]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
import sys
sys.path.insert(0, '../')
from portfolio import *

pd.set_option('display.precision', 3)


## Part 1

### 1(a)

In [169]:
rfr = pd.read_excel('midterm_A_data.xlsx', sheet_name = 'risk-free rate').set_index('Date')
factors = pd.read_excel('midterm_A_data.xlsx', sheet_name = 'factors (excess)').set_index('Date')
assets = pd.read_excel('midterm_A_data.xlsx', sheet_name = 'assets (excess)').set_index('Date')
perf = performanceMetrics(assets, annualization = 12)
perf


,Mean,Vol,Sharpe,Min,Max
NoDur,0.087,0.125,0.693,-0.123,0.100
Durbl,0.107,0.301,0.357,-0.330,0.453
Manuf,0.087,0.174,0.502,-0.208,0.173
Enrgy,0.100,0.246,0.406,-0.346,0.324
HiTec,0.072,0.238,0.304,-0.263,0.192
Telcm,0.023,0.182,0.129,-0.163,0.212
Shops,0.083,0.158,0.527,-0.152,0.180
Hlth,0.079,0.142,0.553,-0.110,0.134
Utils,0.092,0.145,0.630,-0.131,0.112
Other,0.066,0.184,0.356,-0.203,0.166


### 1(b)

In [170]:
perf_sorted = perf.sort_values('Mean', ascending = False)
print('Best Mean: '+ str(round(perf_sorted.iloc[[0]].Mean[0], 3)) + ' for ' + perf_sorted.iloc[[0]].index[0])
perf_sorted = perf.sort_values('Sharpe', ascending = False)
print('Best Sharpe Ratio: '+ str(round(perf_sorted.iloc[[0]].Sharpe[0], 3)) + ' for ' + perf_sorted.iloc[[0]].index[0])

Best Mean: 0.107 for Durbl
Best Sharpe Ratio: 0.693 for NoDur


### 2

In [171]:
tail = tailMetrics(assets)
tail

,Skewness,Kurtosis,VaR (0.05),CVaR (0.05),Max Drawdown,Peak,Bottom,Recover,Duration (to Recover)
NoDur,-0.539,0.983,-0.057,-0.080,-0.350,2007-11-30,2009-02-28,2010-09-30,1035 days
Durbl,0.926,5.200,-0.117,-0.161,-0.767,2000-04-30,2009-02-28,2013-07-31,4840 days
Manuf,-0.577,1.923,-0.085,-0.116,-0.549,2007-10-31,2009-02-28,2011-02-28,1216 days
Enrgy,0.045,3.964,-0.107,-0.147,-0.664,2014-06-30,2020-03-31,2022-03-31,2831 days
HiTec,-0.507,1.418,-0.121,-0.160,-0.813,2000-03-31,2002-09-30,2017-07-31,6331 days
Telcm,-0.228,1.346,-0.095,-0.122,-0.774,2000-03-31,2009-02-28,2017-01-31,6150 days
Shops,-0.152,1.039,-0.075,-0.097,-0.409,2007-05-31,2009-02-28,2010-12-31,1310 days
Hlth,-0.309,0.289,-0.072,-0.088,-0.426,2000-12-31,2009-02-28,2012-04-30,4138 days
Utils,-0.691,1.057,-0.065,-0.097,-0.392,2007-10-31,2009-02-28,2011-10-31,1461 days
Other,-0.534,2.007,-0.084,-0.126,-0.696,2007-05-31,2009-02-28,2014-10-31,2710 days


### 3

In [184]:
cm = assets.corr()
cm[cm==1] = None
cm_ranked = abs(cm.unstack()).sort_values(ascending = False).dropna()
display(cm)
print('Highest correlation: ' + str(cm_ranked.index[1]))
print('Lowest correlation: ' + str(cm_ranked.index[-1]))

,NoDur,Durbl,Manuf,Enrgy,HiTec,Telcm,Shops,Hlth,Utils,Other
NoDur,NaN,0.532,0.740,0.502,0.452,0.621,0.670,0.616,0.575,0.712
Durbl,0.532,NaN,0.777,0.469,0.636,0.588,0.730,0.489,0.342,0.760
Manuf,0.740,0.777,NaN,0.638,0.703,0.704,0.785,0.633,0.498,0.882
Enrgy,0.502,0.469,0.638,NaN,0.375,0.473,0.424,0.392,0.475,0.589
HiTec,0.452,0.636,0.703,0.375,NaN,0.712,0.696,0.527,0.267,0.660
Telcm,0.621,0.588,0.704,0.473,0.712,NaN,0.685,0.567,0.378,0.712
Shops,0.670,0.730,0.785,0.424,0.696,0.685,NaN,0.590,0.385,0.788
Hlth,0.616,0.489,0.633,0.392,0.527,0.567,0.590,NaN,0.450,0.632
Utils,0.575,0.342,0.498,0.475,0.267,0.378,0.385,0.450,NaN,0.459
Other,0.712,0.760,0.882,0.589,0.660,0.712,0.788,0.632,0.459,NaN


Highest correlation: ('Manuf', 'Other')
Lowest correlation: ('Utils', 'HiTec')


## Part 2

### 1

In [173]:
def tangency_portfolio(data, return_vector, inverse_covmat):
    one_vector = np.ones((10,1), dtype = int)
    return np.dot(inverse_covmat, return_vector) / float(np.dot(np.dot(one_vector.transpose(), inverse_covmat), return_vector))

return_vector = np.array(perf['Mean']).reshape([10,1])

covmat = assets.cov()*12
inverse_covmat = np.linalg.inv(np.array(covmat))

tangency_portfolio_weights = tangency_portfolio(assets, return_vector, inverse_covmat)
pd.DataFrame(tangency_portfolio_weights, index = perf.index, columns = ['Weights for the Tangency Portfolio']).sort_values('Weights for the Tangency Portfolio', ascending = False).style.format('{:,.2%}')

,Weights for the Tangency Portfolio
NoDur,74.75%
Shops,42.92%
Utils,29.65%
Hlth,28.27%
HiTec,16.23%
Enrgy,10.91%
Manuf,6.62%
Durbl,1.60%
Other,-49.59%
Telcm,-61.36%


### 2(a)

In [174]:
def tangency_portfolio_allocation(data, return_vector, inverse_covmat, target_return = 0.01*12):
    one_vector = np.ones((10,1), dtype = int)
    return (np.dot(np.dot(one_vector.transpose(), inverse_covmat), return_vector) / np.dot(np.dot(return_vector.transpose(), inverse_covmat), return_vector)) * target_return

scaler = tangency_portfolio_allocation(assets, return_vector, inverse_covmat, target_return = 0.01*12)[0][0]
portfolio_weights = tangency_portfolio_weights * scaler
pd.DataFrame(portfolio_weights, index = perf.index, columns = ['Weights for the Target Portfolio']).sort_values('Weights for the Target Portfolio', ascending = False).style.format('{:,.2%}')

,Weights for the Target Portfolio
NoDur,67.44%
Shops,38.72%
Utils,26.74%
Hlth,25.51%
HiTec,14.64%
Enrgy,9.84%
Manuf,5.97%
Durbl,1.45%
Other,-44.74%
Telcm,-55.35%


### 2(b)

In [185]:
print("Total weight of target portfolio:", "{:.3f}".format(scaler))

Total weight of target portfolio: 0.902


Total weight of target portfolio 0.902 < 1, so the rest is invested in the risk-free rate

### 3

In [193]:
mean = np.dot(return_vector.transpose(), portfolio_weights)
volatality = np.dot(np.dot(portfolio_weights.transpose(), covmat), portfolio_weights) ** 0.5
print(f'Mean = {round(float(mean)*100, 2)}%')
print(f'Volatility = {round(float(volatality)*100, 2)}%')
print(f'Sharpe Ratio = {round(float(mean/volatality), 2)}')


Mean = 12.0%
Volatility = 12.24%
Sharpe Ratio = 0.98


### 4

One thing that should be considered is Bitcoin's correlation with the rest of the assets. If Bitcoin has low correlation with the rest of the assets, then it expands the investment opportunity set (diversifies) and should be considered a separate asset.

### 5

Ideally, we should use the expectation of returns going forward. In our optimisation, we replaced the expectation of future returns with historic data, under the assumption that historic data is a good estimate. However, in the HW1 case, HMC predicted that the mean return in the future will decrease while volatilities will increase. Therefore, in this case, historic data is not a good predictor of future returns, which causes our optimal portfolio based on historic data to underperform in the future.

## Part 3

### 1

In [177]:
mfm_ts = get_ols_metrics(factors, assets, annualization = 12, ignorenan = True)
display(mfm_ts.style.format(formatter = {'alpha' : '{:,.2%}', 'r-squared' : '{:,.2%}','MKT' : '{:,.2f}',\
                                         'HML' : '{:,.2f}', 'RMW' : '{:,.2f}', 'Info Ratio' : '{:,.2f}'}))

,alpha,MKT,HML,RMW,r-squared,Info Ratio
NoDur,1.78%,0.66,0.13,0.41,63.82%,0.24
Durbl,-1.91%,1.59,0.20,0.33,63.84%,-0.11
Manuf,-0.38%,1.08,0.19,0.29,88.13%,-0.06
Enrgy,0.85%,0.99,0.65,0.14,47.57%,0.05
HiTec,3.07%,1.22,-0.48,-0.41,89.18%,0.39
Telcm,-3.57%,0.94,0.02,-0.03,66.51%,-0.34
Shops,0.42%,0.94,-0.05,0.37,76.10%,0.05
Hlth,3.36%,0.66,-0.09,0.09,51.36%,0.34
Utils,3.73%,0.52,0.15,0.30,30.03%,0.31
Other,-2.11%,1.08,0.51,0.03,92.01%,-0.40


### 2

In [178]:
alpha_abs = mfm_ts.abs()
alpha_sorted = alpha_abs.sort_values('alpha')
print("Lowest absolute value of alpha:", alpha_sorted.iloc[[0]].index[0])

Lowest absolute value of alpha: Manuf


Fit of a pricing model is determined by how close alpha is to 0. In this case, Manuf has the alpha closest to 0 and is therefore the best fit.

### 3

We know nothing about the r-squared values, since CAPM says nothing about distribution <br>
Treynor Ratio for all assets would be the same, and equal to the market premium <br>
Info Ratio would be 0, since Info Ratio is proportional to alpha but alpha is 0 given CAPM is a perfect fit

### 4

Not necessarily. CAPM fitting perfectly means that the expectation of the returns of an asset can be modelled by beta * expectation of return on market portfolio + epsilon, where the expectation of epsilon is 0. However, epsilon is a function representing factors beyond the market portfolio that isn't necessarily always 0, so the 3-factor regression may attempt to use the other two factors to explain epsilon.

## Part 4

### 1

In [179]:
rfr_lag = rfr.shift().dropna()
mkt = factors[['MKT']]

In [180]:
forecasts = get_ols_metrics(rfr_lag, mkt)
forecasts

,alpha,RF,r-squared,Treynor Ratio,Info Ratio
MKT,0.011,-4.634,0.023,-0.001,0.251


### 2

In [181]:
w = rfr['RF']
w = w.apply(lambda x: 100 * forecasts['alpha'] + x * 100 * forecasts['RF'])
print("Final value of time series:", "{:.3f}".format(w.iloc[-1]['MKT']))

Final value of time series: 0.985


### 3

In [182]:
strat = w.shift().mul(mkt).rename(columns = {'MKT': 'strat'})
perf_strat = performanceMetrics(strat_lag, annualization = 12)
perf_strat

,Mean,Vol,Sharpe,Min,Max
MKT,0.086,0.144,0.598,-0.17,0.147


### 4

In [183]:
get_ols_metrics(mkt, strat, annualization = 12)

,alpha,MKT,r-squared,Treynor Ratio,Info Ratio
strat,0.059,0.519,0.355,0.181,0.541


### 5

Short the risk-free rate by 4.634 times the overall market. This way, the return becomes alpha + epsilon, which is no longer subject to the volatility of the market.